## 路径还原-模糊匹配

In [44]:
include("../src/CCKS-mKGQA.jl")
include("../src/tools/regexpr.jl") # 正则表达
include("../src/loaddata/extractdata.jl") # 原始数据
include("../src/loaddata/translatedata.jl") # 翻译数据
include("../src/tools/ettalign.jl") # 对齐工具
include("../src/tools/distance.jl") # 距离函数
include("../src/process/submitpath.jl") # 数据提交
include("../src/datatype.jl")
mkpath("submit")

"submit"

In [45]:
illpath = "ILLs_official"
illpath = "ILLs_4"
illpath = "ILLs_3"
illpath = "ILLs_wiki"
# illpath = "ILLs_11"
# illpath = "ILLs_12"
include("../src/loaddata/illsdata.jl");

In [3]:
mt_train_ques = replace.(lowercase.(mt_train_ques), '?' =>"", r"(do|did) you know " => "", r"(.*)/ .*" => s"\g<1>");
mt_valid_ques = replace.(lowercase.(mt_valid_ques), '?' =>"", r"(do|did) you know " => "", r"(.*)/ .*" => s"\g<1>");

### 生成结果

In [51]:
submit_id = "deduct-ills11"
sols = split.(strip.(split(strip(read(open("predict/predict_data_0728.txt", "r"), String)), '\n')),'\t')
push!(sols[427], "language")
push!(sols[526], "language")
push!(sols[1283], "language")
push!(sols[986], "profession")
push!(sols[589], "profession")
push!(sols[688], "profession")
push!(sols[1194], "education_level")
noans = "<http://dbpedia.org/resource/Francis_Russell,_Marquess_of_Tavistock>#<http://dbpedia.org/property/parents>#<http://dbpedia.org/resource/John_Russell,_4th_Duke_of_Bedford>#<http://zh.dbpedia.org/resource/第四代贝德福德公爵约翰·罗素>#<http://zh.dbpedia.org/property/successor>#<http://zh.dbpedia.org/resource/丹尼尔·芬奇，第八代温奇尔西伯爵>";
# 返回结果
res = Vector{String}(undef, 1500)
fails, notgood = Int[], Int[]
## 精确匹配
for (i, sol) in enumerate(sols)
    que, rels = ill_valid_ques[i], sol[3:end]
    ner, rawner = ill_valid_ners[i], valid_ners[i]
    # 精确路径
    submits = precise_submit(ner, rels, illsdata)
    isempty(submits) && (push!(fails, i);continue) # 无解
    bettersubmits = filter(contains(rawner), submits) # 不需要实体链接
    isempty(bettersubmits) ? push!(notgood, i) : (submits = bettersubmits)
    res[i] = first(submits)
end
println(length(fails), '\t', length(notgood))
flush(stdout)
res[fails] .= noans
yestoday = copy(res)
## 模糊匹配
vaguefails, vague_inds, vague_res, vaguenotgood = Int[], Int[], String[], Int[]
for (i, sol) in enumerate(sols)
    (i ∉ fails || length(sol) ≤ 3) && continue
    que, rels = ill_valid_ques[i], sol[3:end]
    ner, rawner = ill_valid_ners[i], valid_ners[i]
    # 精确路径
    submits = vague_submit(ner, rels, illsdata)
    isempty(submits) && (push!(vaguefails, i);continue) # 无解
    bettersubmits = filter(contains(rawner), submits) # 不需要实体链接
    isempty(bettersubmits) ? push!(vaguenotgood, i) : (submits = bettersubmits)
    push!(vague_res, first(submits))
    push!(vague_inds, i)
    res[i] = first(submits)
end
println(length(fails) - length(vague_inds), '\t', length(vaguenotgood))

64	7
34	1


In [49]:
open("fails.txt", "w") do io 
    for i in fails
        println(io, i, '\t', mt_valid_ques[i], '\t', join(sols[i][3:end], '\t'))
    end
end

### 问题纠正

In [78]:
ind = 193
println(valid_ques[ind])
println(mt_valid_ques[ind])
println(join(sols[ind][3:end], '\t'))

BEST_(After_School_album)的创作者做什么职业的
what do the creators of NER do?
title	note


In [53]:
rels = Dict(
    83  => ("chieftown", "canton"),
    193 => ("chronology", "profession"),
    213 => ("title", "lyrics", "profession"),
    231 => ("regent", "issue", "title"),
    93 => ("club", "captain", "countryofbirth"),
    234 => ("teams", "founder", "birth_place"),
    392 => ("lyrics", "before", "nationality"),
    480 => ("club", "captain", "countryofbirth"),
    421 => ("chronology", "profession"),
    503 => ("title", "first", "awards"),
    705 => ("ship_namesake", "eponym", "issue"),
    858 => ("after", "producer", "club"),
    1026 => ("club", "captain", "profession"),
    1058 => ("parent", "east", "timezone"),
    1085 => ("title", "first", "battles"),
    1343 => ("s", "timezone"),
    # 1421 => ("sponsor", "starring"),
    1434 => ("chieftown", "canton"),
    1465 => ("city", "named_for", "combatant"),
    1428 => ("lyrics", "before", "profession")
)
for (ind, rel) in rels
    subm = precise_submit(mt_valid_ques_ner[ind][2], rel, illsdata)
    isempty(subm) ? (subm = vague_submit(mt_valid_ques_ner[ind][2], rel, illsdata)) : println("precise $ind")
    res[ind] = subm[1]
end

precise 392
precise 213
precise 1085
precise 503
precise 1428
precise 1465
precise 480
precise 93
precise 231


In [54]:
submit_io = open("submit/submit_$submit_id.txt", "w")
demo_io = open("submit_test/demo_$submit_id.txt", "w")
println(submit_io, "id\tans_path")
for (i, sol) in enumerate(res)
    println(submit_io, i-1, '\t', sol)
    info = triple_info.(partition(split(sol, '#'), 3))
    sol == noans && continue
    println(demo_io, i-1, '\t', join(mt_valid_ques_ner[i], '\t'))
    println.(Ref(demo_io), info)
    println(demo_io)
end
close(submit_io)
close(demo_io)

### 问题纠正

In [ ]:
"provincial capital of a person or thing" => "seat"

In [48]:
dict_precise_ques = DefaultDict{String, Vector{Tuple}}(Vector{Tuple})
for (que, rels) in zip(mt_train_ques, mt_train_rels)
    que = replace(lowercase(que), '?' => "")
    Tuple(rels) ∈ dict_precise_ques[que] && continue
    push!(dict_precise_ques[que], Tuple(rels))
end

In [108]:
haskey(dict_precise_ques, mt_valid_ques[78])

true

In [50]:
validfails = filter(ind -> haskey(dict_precise_ques, mt_valid_ques[ind]), fails)
for ind in validfails
    que, ner = mt_valid_ques[ind], mt_valid_ques_ner[ind][2]
    isempty(precise_submit(ner, dict_precise_ques[que][1], illsdata)) || println(ind)
end

842


In [ ]:
dict_precise_ques

In [34]:
# ride me
list1 = [427, 526, 986, 688, 1194]
println(all(in(vaguefails), list1))
pushfirst!(list1, 317)
valid_ques[list1]

true


6-element Vector{String}:
 "Ride_Me_(album)的头衔的作词者的语言是什么"
 "Savez-vous quelle langue Paroles de Le titre de Ride_Me_(album) utilise"
 "Savez-vous quelle langue Paroles de Le titre de Ride_Me_(album) utilise"
 "Ride_Me_(album)的头衔的作词者做什么职业的"
 "what is the job of the lyrics of the title of Ride_Me_(album)?"
 "where is the lyrics of the title of Ride_Me_(album) educated?"

In [45]:
dicts = readtuples("analyse/dict_men2rel_tmp.txt"; size=2)
push!(dicts, ("important positions", "office"))
filter(i->i[1]=="important positions", dicts)

1-element Vector{Tuple{String, String}}:
 ("important positions", "office")

### 数据对比

In [56]:
# 数据比对
cmp = open("submit_test/compare_wiki_ills3.txt", "w")
i = j = 0
name1, name2 = "wiki", "ills3"
for (k, que, ner, res1, res2) in zip(1:1500, mt_valid_ques, valid_ners, wiki, ills3)
    res1 == res2 && continue
    println(cmp, k, '\t', que, '\t', ner)
    if res1 == noans
        println(cmp, "$name1\nNONE\n$name2")
        println.(Ref(cmp), triple_info.(partition(split(res2, '#'), 3)))
        j += 1
    elseif res2 == noans
        println(cmp, "$name1")
        println.(Ref(cmp), triple_info.(partition(split(res1, '#'), 3)))
        println(cmp, "$name2\nNONE")
    else
        println(cmp, "$name1")
        println.(Ref(cmp), triple_info.(partition(split(res1, '#'), 3)))
        println(cmp, "$name2")
        println.(Ref(cmp), triple_info.(partition(split(res2, '#'), 3)))
    end
    println(cmp)
    i += 1
end
close(cmp)
println("数据差异 $i, $name2 新解数 $j")

数据差异 260, ills3 新解数 2


In [ ]:
453
257
260

In [45]:
(1500 - 300)/1500

0.8